# One big biome table

Since I am hoping to make comparisons of wood and leaf endophyte environmental patterns, I need to combine these datasets early in the biomformatics pipeline, to make them as comparable as possible. We'll try to stick to the [usearch (uparse)](http://drive5.com/usearch/) pipeline for the process, as much as possible.

------------

That was a year ago. 

Since then my car has been stolen, with my laptop in it. And the manuscripts which are based on these manuscripts are submitted. Somehow, the versions of this notebook and the following analysis notebook that were on github lost most of its graphic outputs, charts and maps and stuff. While most of the important graphics, etc, were backed up, I lost some of the computationally expensive intermediate files necessary to repopulate. 

So to make this notebook useable to reviewers and readers, I'll be picking through this process again, and the downstream analysis, if I don't go insane in the meantime. Maybe even if I do.

## Table of contents

[Work environment](#environment)

[Rearranging barcodes](#rearrange)

[Trimming reads](#trim)

[Merging paired-end reads](#mergepairs)

[Quality filtering](#qf)

[Convert form fastq to fasta format](#fastq2fasta)

[Convert form fastq to fasta format](#fastq2fasta)

[Demultiplex leaf reads](#demult)

[Remove primers](#removeprimers)

[Floating primers](#defloat)

[Chimera checking](#chimera)

[Combining fasta files from all 3 studies](#combine)

[OTU clustering](#otuclust)

[Customizing UNITE database](#UNITE)

[Assign taxonomy](#asstax)

[Make and tidy up biom table](#makebiom)

[Add metadata](#addmetadata)

<a id='environment'></a>

### Work environment

Working directory, on my machine:

In [2]:
cd /home/daniel/Documents/Taiwan_data/combined/combo_biome

We'll be using the [usearch (uparse)](http://drive5.com/usearch/) pipeline, version v8.0.1623_i86linux32, on the University of Oregon's Talapas computing cluster.

<a id='rearrange'></a>
### Rearranging barcodes

We need to merged paired end sequences of the leaves and wood. But before we can do this, there are several steps. First, the leaf study reads include a split 6+6 bp barcode scheme for identifying reads, so these need to be clipped from one read and combined on the other. I wrote a python script for this:

In [4]:
cat scripts/BCunsplit.py

#!/usr/bin/env python3

## lets try to take a two unpaired read files, cut out the bp from the reverse,
## and tack it onto the forward.
## have to preserve the fastq format so that pandaseq can do unsplit3.py forward_reads reverse_reads

#The first six bps and quality ratings of the reverse reads should be chopped off and placed after
#the first six bps of the forward reads and quality ratings. For use with fastq files. It will spit
#out two files, with the names: "rearranged_[your orignial forward and reverse read file names].fastq".

import itertools ##to let us jump around 
from sys import argv

script, forward_file, reverse_file = argv

forwardlabels=[]
reverselabels=[]
forwardreads=[]
forwardreadsq=[]
reversereads=[]
reversereadsq=[]
forwardBC=[]
forwardBCq=[]
reverseBC=[]
reverseBCq=[]

with open(forward_file) as foop:

	#labels:

	for h in itertools.islice(foop, 0, None, 4):
		forwardlabels.append(h)

##forward sequencies, barcodes: 

	foop.seek(0)
    
	for i in itertools.isli

I have details on how I used this [here](https://github.com/danchurch/taiwan_dada2/blob/master/dada2pipeline.ipynb). 

This outputs two files, "rearranged_Roo_R2.fastq" and "rearranged_Roo_R2.fastq". I did this in another directory, so we'll add some sym links here for convenience:

In [8]:
## leaves
ln -s /home/daniel/Documents/taiwan/taiwan_dada2/rearranged_leafR1.fastq reLeafR1.fastq
ln -s /home/daniel/Documents/taiwan/taiwan_dada2/rearranged_leafR2.fastq reLeafR2.fastq

<a id='trim'></a>
### Trimming reads

Next we trim a little to make sure we doing our alignments with high quality base calls. The sites for trimming are decided by looking at the raw reads [(see below)](#quality), and finding where quality begins to drop off. 
To trim, we'll use the [FASTX-toolkit](http://hannonlab.cshl.edu/fastx_toolkit/).

Our wood reads are already demultiplexed, so we don't have a single forward and reverse read file for all of our wood samples, like we do above with the leaves. So let's make a script for this:

In [ ]:
## trims.sh
#####################################################


## wood reads live here:
wooddir=/home/daniel/Documents/taiwan/woodreads/

## working directory is here:
cd /home/daniel/Documents/taiwan/taiwan_combined_biom

###### R1 reads:

## home for trimmed R1 wood reads here:
R1trimdir='/home/daniel/Documents/taiwan/taiwan_combined_biom/trimmed_wood/R1/'

## trim just the R1s, output to their new home with new filename:
for i in $wooddir*_R1_*; do
    echo $i
    out=$R1trimdir$(basename ${i/_001\.fastq/_trimmed\.fastq})
    fastx_trimmer -l 255 -i $i -o $out && echo $out 
done

###### R2 reads:

## home for trimmed R2 wood reads here:
R2trimdir='/home/daniel/Documents/taiwan/taiwan_combined_biom/trimmed_wood/R2/'

## trim just the R2s, output to their new home with new filename:
for j in $wooddir*_R2_*; do
    echo $j
    out=$R2trimdir$(basename ${j/_001\.fastq/_trimmed\.fastq})
    fastx_trimmer -l 210 -i $j -o $out && echo $out 
done

In [2]:
## leaves. These lengths were decided by Roo. They are all still in one pile:
fastx_trimmer -l 263 -i reLeafR1.fastq -o Roo_R1_trimmed.fastq
fastx_trimmer -l 170 -i reLeafR2.fastq -o Roo_R2_trimmed.fastq

<a id='mergepairs'></a> 

### Merging paired-end reads

In [ ]:
#!usr/bin/env bash

#SBATCH --job-name=merge_wood
#SBATCH --output=merge_wood.out
#SBATCH --error=merge_wood.err
#SBATCH --time=0-05:00:00
#SBATCH --nodes=1

module load usearch/8.0

cd projects/xylaria/dthomas/

R1d=/projects/xylaria/dthomas/trimmed_wood/R1/

for forward in $R1d*; do
    echo $forward
    reverse=${forward//R1/R2}
    aa=$(basename $forward); output="/projects/xylaria/dthomas/merged_wood/"${aa/_R1_trimmed.fastq/_merged.fastq}
    usearch -fastq_mergepairs $forward -reverse $reverse -fastqout $output
done
